In [16]:
#Import training data from IBM Cloud
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': ,
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token',
    'api_key': 
}

configuration_name = 
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_train = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('train.csv', 'bostonhousing-donotdelete-pr-f7plktwicr9r0f'), inferSchema='true')
#df_data_train.take(1)


In [17]:
#Import the test set to be submitted to Kaggle as Spark dataframe. Use schema from the train set.
df_test = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('test.csv', 'bostonhousing-donotdelete-pr-f7plktwicr9r0f'), schema=df_data_train.schema)
#df_test.take(1)


#Import the sample submission for Kaggle as pandas dataframe
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=,
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_c63666ea1c9d403e811d4c86742ad623.get_object(Bucket='bostonhousing-donotdelete-pr-f7plktwicr9r0f',Key='sample_submission.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

sample_sub = pd.read_csv(body)
sample_sub.head()


,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [18]:
df = df_data_train


In [19]:
#Replace the PoolArea feature with a Has Pool feature, there are very few houses with pools in the dataset (only 7 of 1460)
from pyspark.ml.feature import Binarizer

df = df.withColumn("PoolArea", df.PoolArea.cast("double"))

binarizer = Binarizer(threshold=0.0, inputCol='PoolArea', outputCol='Has_Pool')

df = binarizer.transform(df)

df = df.drop('PoolArea')

In [20]:
df.printSchema()



root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

In [21]:
#Create two separate lists of categorical and numerical features
columns_cat = [item[0] for item in df.dtypes if item[1].startswith('string')]
columns_num = [item[0] for item in df.dtypes if item[1].startswith('int') | item[1].startswith('double')]

print(columns_cat)
print(columns_num)

['MSZoning', 'LotFrontage', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', '

In [22]:
#Find correlation between numerical features and the sale price

from operator import itemgetter

corr_list = []

for column in columns_num:
    corr_val = df.corr('SalePrice', column)
    corr_list.append((column, corr_val))

corr_list = sorted(corr_list, key=itemgetter(1))

print(corr_list)

num_cols = []

#Create list of numerical cols with a correlation above 0.4 with the sale price
for col, val in corr_list:
    if col == 'SalePrice':
        continue
    if val > 0.4:
        num_cols.append(col)

print(num_cols)
    
    

[('KitchenAbvGr', -0.13590737084214102), ('EnclosedPorch', -0.1285779579259566), ('MSSubClass', -0.08428413512659509), ('OverallCond', -0.07785589404867796), ('YrSold', -0.02892258516873857), ('LowQualFinSF', -0.02560613000067957), ('Id', -0.021916719443430967), ('MiscVal', -0.021189579640303206), ('BsmtHalfBath', -0.016844154297358943), ('BsmtFinSF2', -0.011378121450215155), ('3SsnPorch', 0.04458366533574838), ('MoSold', 0.0464322452238194), ('Has_Pool', 0.0937081251255179), ('ScreenPorch', 0.11144657114291114), ('BedroomAbvGr', 0.1682131543007397), ('BsmtUnfSF', 0.21447910554696925), ('BsmtFullBath', 0.22712223313149418), ('LotArea', 0.2638433538714051), ('HalfBath', 0.2841076755947826), ('OpenPorchSF', 0.31585622711605504), ('2ndFlrSF', 0.3193338028320674), ('WoodDeckSF', 0.32441344456812926), ('BsmtFinSF1', 0.38641980624215344), ('Fireplaces', 0.46692883675152763), ('YearRemodAdd', 0.5071009671113869), ('YearBuilt', 0.522897332879497), ('TotRmsAbvGrd', 0.5337231555820284), ('FullBa

In [23]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

cat_cols = ['MSZoning']

# Convert categorical strings to index values
indexer = StringIndexer(inputCol='MSZoning', outputCol='MSZoning_idx')

# One-hot encode index values
onehot = OneHotEncoderEstimator(
    inputCols=['MSZoning_idx'],
    outputCols=['MSZoning_dummy']
)

inpcols = num_cols + ['MSZoning_dummy']


# Assemble predictors into a single column 
assembler = VectorAssembler(inputCols=num_cols, outputCol='features')

vect_df = assembler.transform(df)

vect_df.take(1)

h_train, h_test = vect_df.randomSplit([0.8, 0.2], seed=1)



In [24]:
# Create Gradient boosted tree regressor
gbt = GBTRegressor(featuresCol='features',
                           labelCol='SalePrice',
                           seed=1
                           )

gbt_model = gbt.fit(h_train)

gbt_predictions = gbt_model.transform(h_test)

gbt_predictions.select('prediction', 'SalePrice', 'features').show(5)

gbt_evaluator = RegressionEvaluator(
    labelCol='SalePrice', predictionCol='prediction', metricName='rmse')
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)




+------------------+---------+--------------------+
|        prediction|SalePrice|            features|
+------------------+---------+--------------------+
|301581.74510140327|   250000|[1.0,2000.0,2000....|
| 132729.2006719106|   129500|[0.0,1965.0,1965....|
|148501.53603642609|   159000|[0.0,2004.0,2004....|
|304574.62705241435|   325300|[1.0,2006.0,2005....|
|141591.16783698555|   154000|[1.0,2001.0,1968....|
+------------------+---------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 37491


In [25]:
vect_df_test = assembler.transform(df_test)

kaggle_pred = gbt_model.transform(vect_df_test)

kaggle_pred.select('prediction', 'Id').show(5)




+------------------+----+
|        prediction|  Id|
+------------------+----+
|121649.39695095464|1461|
|155125.62015627173|1462|
|153237.37044571713|1463|
| 181912.0984065436|1464|
|  220514.675528486|1465|
+------------------+----+
only showing top 5 rows

